In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("List available databases and tables").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session

spark.catalog.listDatabases()

[Database(name='2022_10_22', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='2023_04_01', catalog='spark_catalog', description='FactSet data version for the day', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse'),
 Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://bialobog.cs.ucl.ac.uk:8020/user/hive/warehouse')]

In [2]:
from pyspark.sql.functions import regexp_replace
from pyspark.sql import functions as F

spark.sql("USE 2023_04_01")


DataFrame[]

In [3]:
from Boruta import BorutaPy

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier

def boruta_fs(df):
    df = df.toPandas()
    print(df.head())
    exclude_columns = ['fsym_id', 'label']
    df = df[df.loc[:, ~df.columns.isin(exclude_columns)].apply(lambda row: all(len(cell) == 22 for cell in row), axis=1)]
    print("Number of records: ", len(df))
    
    X = df.drop(exclude_columns, axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    model = xgb.XGBClassifier()
    feat_selector = BorutaPy(model, n_estimators='auto', verbose=2, random_state=1)
    feat_selector.fit(X_train, y_train)
    
    print(feat_selector.support_)
    print(feat_selector.ranking_)
    

In [4]:
from CreateDataset import get_full_series_stocks, get_features_all_stocks_seq

df = pd.read_csv('imploded_stocks_price.csv', index_col=False)
full_series_stocks = get_full_series_stocks(df) #this gets the stocks that have data since 2001
filtered_df = df[df['fsym_id'].isin(full_series_stocks)]
filtered_df = get_features_all_stocks_seq(filtered_df)

boruta_fs(filtered_df)

/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


File found


/home/ztewari/Stock-Implosion-Prediction-FYP/CreateDataset.py:155: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date_for_year'] = df['fsym_id'].map({k: list(generate_dates()) for k, g in df.groupby('fsym_id')})


    fsym_id                                       ff_assets_gr  \
0  B01DPB-R  [-0.014595571478033439, -0.010777702513382885,...   
1  B04HL7-R  [-0.017397073496077105, -0.020468159238539684,...   
2  B0592Z-R  [-0.22886968060721727, -0.06601791095714236, 0...   
3  B14762-R  [-0.032416479717869116, -0.010892328172492517,...   
4  B14DJ7-R  [-0.03358909949090103, -0.030525608273185175, ...   

                                         ff_earn_yld  \
0  [0.02092100143201426, 0.021141620914616178, 0....   
1  [0.02122850411385511, 0.021808476383660094, 0....   
2  [0.019967765951548373, 0.020620576308470057, 0...   
3  [0.021455647121493452, 0.02170320233551796, 0....   
4  [0.02177414232478644, 0.02119885143432527, 0.0...   

                           ff_net_inc_basic_aft_xord  \
0  [0.04957930125973168, 0.018614827674471292, -0...   
1  [-0.20109244752845495, -0.16673331652565856, -...   
2  [-0.1861407339251776, -0.23067811361755566, -0...   
3  [-0.15083260615478794, -0.1375070243133

ValueError: setting an array element with a sequence.